In [1]:
from src.utils.hybridsearch import run_hybrid_search

res = run_hybrid_search(
    query="có ip 15 512 ko",
    k=5
)

NEO4J_URI: bolt://localhost:7687
USERNAME: neo4j
DATABASE: neo4j
EMBEDDINGS_MODEL: None


In [2]:
res

[Document(metadata={'id': '82d5002647f789d4ccf8e08621f6acf5', 'source': 'data/hoanghamobile_with_summary.csv', '_id': '666baeb69793e149fe739409', 'url': 'https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-15-512gb-chinh-hang-vn-a', 'row': 87}, page_content="\ntext: combined_info: Tên sản phẩm: điện thoại iphone 15 (512gb) - chính hãng vn/a | Khuyến mãi: - KM 1<br>- Giảm thêm 100.000đ khi khách hàng thanh toán bằng hình thức chuyển khoản ngân hàng khi mua iPhone 15 Series.<br>- KM 2<br>- Ưu đãi trả góp 0% qua thẻ tín dụng<br> | Thông số kỹ thuật: Công nghệ màn hình:\nMàn hình Super Retina XDR, Tấm nền OLED, Dynamic Island, Màn hình HDR, Tỷ lệ tương phản 2.000.000:1 , Màn hình có dải màu rộng (P3), Haptic Touch<br> Độ phân giải:\n1179 x 2556, Chính: 48MP, khẩu độ ƒ/1.6, Ultra Wide: 12MP, khẩu độ ƒ/2.4, Camera trước TrueDepth 12MP, khẩu độ ƒ/1.9<br> Kích thước màn hình:\n6.1 inch<br> Hệ điều hành:\niOS 17<br> Vi xử lý:\nA16 Bionic<br> Bộ nhớ trong:\n512GB<br> RAM:\n6GB<br> Mạng di 